In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f229ce40d7f898400f8498ae31f886f586780d4c59b16d6e2222ced82e713fa6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [78]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [79]:
data = [
    (1,11,2),(1,11,3),(2,11,3),
    (3,11,2),(3,11,4),(4,11,1),
    (4,11,2),(4,11,3),(4,12,5),
    (5,12,1),(5,12,2),(5,12,6)
    ]

**Parte 1**

1.1 Precalentamiento: Triángulos.



**Fase de Map.**

Cada mapper tendra una cierta cantidad de aristas del grafo. Para cada arista (n1, R, n2),
el mapper debe hacer lo siguiente.

1. Hashear los nodos: b1 = h(n1), b2 = h(n2).
2. Como el triangulo que formemos solo puede mapear la variable x a n1 e y a n2, la arista (n1, R, n2)
solo podrıa formar parte de un trıangulo de la forma (n1, n2, a) para un a ∈V . Entonces, generamos b
llaves (b1, b2, 0), . . . , (b1, b2, b −1), y emitimos b mensajes con el contenido de la arista, uno para cada
llave: ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))

Para la primera parte, realizaremos un una función que permita hashear de la forma (key, arista)

In [80]:
def key_edge(edge, b):
  map = [] # Lista que contiene los pares key, arista para una determinada edge
  n1, label, n2 = edge # Descomponemos la arista entregada
  for i in range (b): #Iteramos en los valores de mod 2: 0,1
    map.append(((n1%b, n2%b, i), edge)) #agregamos los pares ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))
  return map



**Probamos para Triangulos (X,11,Y), (Y,11,Z), (Z,11,X)**

In [81]:
data_triangles = [edge for edge in data if edge[1] == 11]

rdd = sc.parallelize(data_triangles)
rdd.collect()


[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3)]

In [82]:
map_rdd = rdd.flatMap(lambda x: key_edge(x, 2))
map_rdd.collect()

[((1, 0, 0), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 0), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 4)),
 ((1, 0, 1), (3, 11, 4)),
 ((0, 1, 0), (4, 11, 1)),
 ((0, 1, 1), (4, 11, 1)),
 ((0, 0, 0), (4, 11, 2)),
 ((0, 0, 1), (4, 11, 2)),
 ((0, 1, 0), (4, 11, 3)),
 ((0, 1, 1), (4, 11, 3))]

**Fase de Reduce.**

 Los reducers reciben siempre los mensajes correspondientes a alguna llave (b1, b2, b3).
Pero en el valor de esa llave se encuentran aristas. Todas estas aristas forman un pequeño grafo , y el reducer
emite como respuesta todas las tuplas (n1, n2, n3) correspondientes a los triangulos que detecta en su grafo

In [83]:
group_rdd = map_rdd.groupByKey().mapValues(list)
group_rdd.collect()

[((1, 0, 0), [(1, 11, 2), (3, 11, 2), (3, 11, 4)]),
 ((1, 1, 1), [(1, 11, 3)]),
 ((0, 1, 0), [(2, 11, 3), (4, 11, 1), (4, 11, 3)]),
 ((0, 0, 1), [(4, 11, 2)]),
 ((1, 0, 1), [(1, 11, 2), (3, 11, 2), (3, 11, 4)]),
 ((1, 1, 0), [(1, 11, 3)]),
 ((0, 1, 1), [(2, 11, 3), (4, 11, 1), (4, 11, 3)]),
 ((0, 0, 0), [(4, 11, 2)])]

**Reducer**